In [1]:
import numpy as np
import pandas as pd
import reverse_geocoder as rg 
import pprint 
from datetime import datetime,time, date
import requests
import networkx as nx
import json
import matplotlib.pyplot as plt
from math import radians, cos, sin, asin, sqrt
import time

In [2]:
trip_df = pd.read_csv('2016_june_cleaned.csv')

In [3]:
trip_df.shape

(51607, 14)

In [4]:
trip_df = trip_df.rename(columns = {'Unnamed: 0': 'trip_id'})

In [5]:
trip_df.head()

,trip_id,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,dropoff_longitude,dropoff_latitude,payment_type,total_amount,dropoff_lat_long_tuple,dropoff_city
0,0,2016-06-01 00:00:55,2016-06-01 00:31:25,1,19.20,-73.790199,40.646770,2,-73.966484,40.789272,1,66.34,"(40.78927230834961, -73.96648406982422)",Manhattan
1,1,2016-06-01 00:01:35,2016-06-01 00:34:08,1,18.52,-73.789909,40.646751,2,-73.962631,40.801632,1,58.34,"(40.80163192749024, -73.96263122558594)",Manhattan
2,2,2016-06-01 00:01:56,2016-06-01 00:34:33,1,21.37,-73.782372,40.644585,2,-73.979279,40.776905,1,70.01,"(40.77690505981445, -73.97927856445312)",Manhattan
3,3,2016-06-01 00:02:36,2016-06-01 00:33:02,1,16.91,-73.789429,40.646759,2,-73.979111,40.766800,1,63.36,"(40.76679992675781, -73.97911071777342)",Manhattan
4,4,2016-06-01 00:03:54,2016-06-01 00:31:41,1,17.40,-73.789833,40.646851,2,-73.971542,40.762051,1,72.96,"(40.7620506286621, -73.97154235839844)",Manhattan


In [6]:
trip_df['language'] = 1

In [7]:
trip_df['profession'] = 1

In [8]:
v = trip_df['language'] == 1
trip_df.loc[v,'language'] = np.random.choice((0,1), v.sum(), p=(0.5,0.5))

In [9]:
v = trip_df['profession'] == 1
trip_df.loc[v,'profession'] = np.random.choice((0,1), v.sum(), p=(0.5,0.5))

In [10]:
trip_df.tail()

,trip_id,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,dropoff_longitude,dropoff_latitude,payment_type,total_amount,dropoff_lat_long_tuple,dropoff_city,language,profession
51602,51602,2016-06-30 23:56:12,2016-07-01 00:33:46,2,17.60,-73.782288,40.644569,2,-73.980667,40.764435,2,52.80,"(40.76443481445313, -73.98066711425781)",Manhattan,1,0
51603,51603,2016-06-30 23:59:04,2016-07-01 00:58:31,2,18.10,-73.781990,40.644711,2,-73.986450,40.758171,1,69.99,"(40.75817108154297, -73.9864501953125)",Manhattan,0,0
51604,51604,2016-06-30 23:59:20,2016-07-01 00:45:34,1,21.22,-73.790337,40.646629,2,-73.984230,40.779942,2,58.34,"(40.77994155883789, -73.9842300415039)",Manhattan,1,0
51605,51605,2016-06-30 23:59:53,2016-07-01 00:32:22,1,16.70,-73.789238,40.646832,2,-73.972740,40.764446,2,52.80,"(40.76444625854492, -73.97274017333984)",Manhattan,0,0
51606,51606,2016-06-30 23:59:54,2016-07-01 00:39:51,2,17.90,-73.784592,40.648472,2,-73.974915,40.758846,2,58.34,"(40.75884628295898, -73.97491455078125)",Manhattan,1,0


In [11]:
trip_df.tpep_pickup_datetime = trip_df.tpep_pickup_datetime.apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
trip_df.tpep_dropoff_datetime = trip_df.tpep_dropoff_datetime.apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
trip_df = trip_df.sort_values(by = 'tpep_pickup_datetime')
# trip_df.head()

In [12]:
trip_df.head()

,trip_id,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,dropoff_longitude,dropoff_latitude,payment_type,total_amount,dropoff_lat_long_tuple,dropoff_city,language,profession
0,0,2016-06-01 00:00:55,2016-06-01 00:31:25,1,19.20,-73.790199,40.646770,2,-73.966484,40.789272,1,66.34,"(40.78927230834961, -73.96648406982422)",Manhattan,0,1
1,1,2016-06-01 00:01:35,2016-06-01 00:34:08,1,18.52,-73.789909,40.646751,2,-73.962631,40.801632,1,58.34,"(40.80163192749024, -73.96263122558594)",Manhattan,1,1
2,2,2016-06-01 00:01:56,2016-06-01 00:34:33,1,21.37,-73.782372,40.644585,2,-73.979279,40.776905,1,70.01,"(40.77690505981445, -73.97927856445312)",Manhattan,0,0
3,3,2016-06-01 00:02:36,2016-06-01 00:33:02,1,16.91,-73.789429,40.646759,2,-73.979111,40.766800,1,63.36,"(40.76679992675781, -73.97911071777342)",Manhattan,0,1
4,4,2016-06-01 00:03:54,2016-06-01 00:31:41,1,17.40,-73.789833,40.646851,2,-73.971542,40.762051,1,72.96,"(40.7620506286621, -73.97154235839844)",Manhattan,1,1


In [13]:
trip_df.tail()

,trip_id,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,dropoff_longitude,dropoff_latitude,payment_type,total_amount,dropoff_lat_long_tuple,dropoff_city,language,profession
51602,51602,2016-06-30 23:56:12,2016-07-01 00:33:46,2,17.60,-73.782288,40.644569,2,-73.980667,40.764435,2,52.80,"(40.76443481445313, -73.98066711425781)",Manhattan,1,0
51603,51603,2016-06-30 23:59:04,2016-07-01 00:58:31,2,18.10,-73.781990,40.644711,2,-73.986450,40.758171,1,69.99,"(40.75817108154297, -73.9864501953125)",Manhattan,0,0
51604,51604,2016-06-30 23:59:20,2016-07-01 00:45:34,1,21.22,-73.790337,40.646629,2,-73.984230,40.779942,2,58.34,"(40.77994155883789, -73.9842300415039)",Manhattan,1,0
51605,51605,2016-06-30 23:59:53,2016-07-01 00:32:22,1,16.70,-73.789238,40.646832,2,-73.972740,40.764446,2,52.80,"(40.76444625854492, -73.97274017333984)",Manhattan,0,0
51606,51606,2016-06-30 23:59:54,2016-07-01 00:39:51,2,17.90,-73.784592,40.648472,2,-73.974915,40.758846,2,58.34,"(40.75884628295898, -73.97491455078125)",Manhattan,1,0


In [14]:
# trip_df['pickup_date'] = trip_df.tpep_pickup_datetime.apply(lambda date_time: date_time.date())
# trip_df['pickup_time'] = trip_df.tpep_pickup_datetime.apply(lambda date_time: date_time.time())

In [15]:
trip_df = trip_df[(trip_df.tpep_pickup_datetime >= datetime(2016, 6, 10, 8,0,0)) 
                  & (trip_df.tpep_pickup_datetime <= datetime(2016, 6, 10, 22,0,0))]

In [16]:
trip_df.shape

(1290, 16)

In [17]:
all_trips_sum = sum(trip_df.trip_distance)

In [18]:
all_trips_sum

23471.47999999997

In [19]:
total_trips = trip_df.shape[0]

In [20]:
B_hdist = []
averageSpeed = None
B_tuple = None
allRouteDistance = dict()
totalRouteDistance = 0

In [21]:
# Pooling Window = 5 mins

def getGroup(a,b):
    res = trip_df[((trip_df.tpep_pickup_datetime - b).apply(lambda x: (x.total_seconds() <= 300) & 
                                                          (x.total_seconds() > 0))) & (trip_df.trip_id != a)]
    return res.trip_id.values


In [22]:
trip_df["time"]=trip_df.tpep_dropoff_datetime-trip_df.tpep_pickup_datetime;
trip_df["time"] = trip_df["time"].apply(lambda x: x.days*24*60*60+x.seconds)  
trip_df = trip_df[trip_df["time"] != 0]
trip_df["speed"]=trip_df.trip_distance/trip_df["time"]
averageSpeed=trip_df["speed"].mean()
#print(averageSpeed)

In [23]:
def haversine(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * asin(sqrt(a))
    r = 3956  # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [24]:
def timeAtoB(hdist):
    return hdist/averageSpeed;


In [25]:
def lowerBound(B_hdist,timeADest, cur_id, B_tuple):
    res=[]
    for i in range(len(B_tuple)):
        b=B_tuple[i]
        timeBDest=trip_df.tpep_dropoff_datetime[b]-trip_df.tpep_pickup_datetime[b];  
        timeBDest=timeBDest.days*24*60*60+timeBDest.seconds; 
        delay=.20*trip_df.trip_distance[b]
        lval=timeADest+timeAtoB(B_hdist[i])
        rval=timeBDest+delay

        if(lval < rval):
            res.append((cur_id,b))
    return res

In [26]:
def startComputation(B_tuple):
#     print("Calculating all haversine distances for B's") 
    all_result = []

    for i in range(len(B_tuple)):
        rest = B_tuple[:i]
        rest.extend(B_tuple[i+1:])
        cur_id = B_tuple[i]
        timeADest = trip_df.tpep_dropoff_datetime[cur_id] - trip_df.tpep_pickup_datetime[cur_id]
        timeADest = timeADest.days*24*60*60+timeADest.seconds
        hDistA=(haversine(trip_df.pickup_longitude[cur_id],trip_df.pickup_latitude[cur_id],trip_df.dropoff_longitude[cur_id],trip_df.dropoff_latitude[cur_id]))
        B_hdist = []
        for b in rest:
            B_hdist.append(abs(haversine(trip_df.pickup_longitude[b],trip_df.pickup_latitude[b],trip_df.dropoff_longitude[b],trip_df.dropoff_latitude[b])-hDistA)) 
    #     print(B_hdist)

    #     print("Applying lower bound. Find the answer below:")
        result = lowerBound(B_hdist, timeADest, cur_id, rest)
#         print("result---->")
#         print(result)
        all_result.extend(result)
#     print("all_result----->")
#     print(all_result)
    return all_result

In [27]:
def getActualDistance(tuple_list):
    
    result = []    #(drop-off_A_trip_id, drop-off_B_trip_id, actual distance from drop_off_A to drop-off_B)
    
    for pair in tuple_list:
        url = "https://graphhopper.com/api/1/matrix?"
        points = "point="+str(trip_df.loc[pair[0], "dropoff_latitude"])+","+str(trip_df.loc[pair[0], "dropoff_longitude"])
        points += "&point="+str(trip_df.loc[pair[1], "dropoff_latitude"])+","+str(trip_df.loc[pair[1], "dropoff_longitude"])
        url += points
        url+="&type=json&vehicle=car&debug=true&out_array=weights&out_array=times&out_array=distances&key=a53f4d5d-57c6-4cee-920f-6479b82f4f68"
        response = requests.get(url)
        response_dict = json.loads(response.text)
#         print(response.text)
#         print(response_dict['distances'])
        actual_dist = response_dict['distances'][0][1] * 0.000621371    # 1 meter = 0.000621371 miles
#         print(actual_dist)    
        result.append((pair[0], pair[1], actual_dist, trip_df.trip_distance[pair[0]], trip_df.trip_distance[pair[1]], trip_df.language[pair[0]], trip_df.language[pair[1]], trip_df.profession[pair[0]], trip_df.profession[pair[1]]))
    
    return result 
    

In [28]:
def getSocialScore(prof1, prof2, lang1, lang2):
    if lang1== lang2:
        if prof1 == prof2:
            return 1
        else:
            return 0.6
    else:
        return 0.1

In [29]:
def getDistanceSaved(distance1, distance2, routeDistance):
    dist = (distance1+distance2) - routeDistance
    return dist

In [30]:
def getWeight(distSaved, socialScore):
    weight = (1.0*distSaved) + (0.0*socialScore)
    return round(weight,3)

In [31]:
trip_df.head(15)

,trip_id,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,dropoff_longitude,dropoff_latitude,payment_type,total_amount,dropoff_lat_long_tuple,dropoff_city,language,profession,time,speed
16303,16303,2016-06-10 08:00:16,2016-06-10 09:16:56,1,21.23,-73.778374,40.646809,2,-73.976936,40.780071,1,72.92,"(40.78007125854492, -73.97693634033203)",Manhattan,1,0,4600,0.004615
16304,16304,2016-06-10 08:00:49,2016-06-10 09:18:54,1,16.90,-73.790573,40.643551,2,-73.985062,40.758209,2,58.34,"(40.758209228515625, -73.98506164550781)",Manhattan,0,1,4685,0.003607
16305,16305,2016-06-10 08:00:57,2016-06-10 09:17:15,1,18.03,-73.782173,40.644730,2,-73.977654,40.761372,1,70.01,"(40.76137161254883, -73.97765350341797)",Manhattan,0,1,4578,0.003938
16306,16306,2016-06-10 08:01:10,2016-06-10 09:14:33,1,20.50,-73.776772,40.645409,2,-73.965668,40.774300,2,58.34,"(40.77429962158203, -73.96566772460938)",Manhattan,1,1,4403,0.004656
16307,16307,2016-06-10 08:01:20,2016-06-10 09:34:29,1,20.35,-73.776680,40.645279,2,-73.980240,40.767632,1,68.64,"(40.76763153076172, -73.98023986816406)",Manhattan,1,1,5589,0.003641
16308,16308,2016-06-10 08:01:49,2016-06-10 09:24:05,1,18.35,-73.776772,40.645519,2,-73.983376,40.756145,2,58.34,"(40.75614547729492, -73.9833755493164)",Manhattan,1,0,4936,0.003718
16309,16309,2016-06-10 08:02:22,2016-06-10 09:27:20,1,17.59,-73.785828,40.645481,2,-73.980171,40.757820,1,70.01,"(40.75782012939453, -73.98017120361328)",Manhattan,1,1,5098,0.003450
16310,16310,2016-06-10 08:02:25,2016-06-10 09:40:55,1,19.04,-73.776863,40.645168,2,-73.982979,40.755211,2,58.34,"(40.75521087646485, -73.98297882080078)",Manhattan,0,0,5910,0.003222
16311,16311,2016-06-10 08:04:25,2016-06-10 09:33:44,1,18.50,-73.776711,40.645184,2,-73.979233,40.761497,1,69.99,"(40.761497497558594, -73.97923278808595)",Manhattan,1,1,5359,0.003452
16312,16312,2016-06-10 08:07:25,2016-06-10 08:09:28,2,0.20,-73.971596,40.757141,2,-73.973770,40.758087,1,73.34,"(40.75808715820313, -73.97377014160156)",Manhattan,0,0,123,0.001626


In [32]:
trips_covered = set()

total_sum = 0

trips_merged = []
totalRouteDistance = 0


start_time = time.time()
pools_processed = 0;

for i,row in trip_df.iterrows():
    
#     print(trips_covered)
    
#     print("********___" + str(row.trip_id) + "___**********")
#     timeADest = row.tpep_dropoff_datetime - row.tpep_pickup_datetime
#     timeADest = timeADest.days*24*60*60+timeADest.seconds
    a=row.trip_id
    
    if(a in trips_covered):
        continue
#     print("Getting all Bs")
    B_tuple= getGroup(row.trip_id, row.tpep_pickup_datetime)
    B_tuple = [val for val in B_tuple if val not in trips_covered]
    trips_covered.add(a)
    for i in B_tuple:
        trips_covered.add(i)
#     print("B_Tuple")
#     print(B_tuple)
#     print("A")
#     print(a)
    
    if B_tuple == []:
        continue
        
#     print(trip_df.tpep_pickup_datetime[a])
    
    B_tuple.append(row.trip_id)
    tuple_list = startComputation(B_tuple)
    actual_dist_values = getActualDistance(tuple_list)
    actual_dist_values     # in miles
    if(len(actual_dist_values) != 0):
        T = np.append(a, B_tuple).tolist()
        linkWeights = []
        for row in actual_dist_values:
            distanceA = row[3]
            distanceB = row[4]
            routeDistance = row[2] + row[3]
            prof1 = row[7]
            prof2 = row[8]
            lang1 = row[5]
            lang2 = row[6]
            allRouteDistance[row[0],row[1]] = routeDistance
            
            socialScore = getSocialScore(prof1, prof2, lang1, lang2)
            distanceSaved = getDistanceSaved(distanceA, distanceB, routeDistance)
            if distanceSaved >= 0:
                finalWeight = getWeight(distanceSaved, socialScore)
                linkWeights.append((row[0], row[1], finalWeight))
        linkWeights.sort(key = lambda x : list(x)[2])
#         print(linkWeights)
        if (len(linkWeights) != 0):
            G = nx.Graph()
            G.add_nodes_from(T)
            for link in linkWeights:
                G.add_edge(link[0], link[1], weight=link[2])

#             nx.draw(G, with_labels=True)
            M_max = nx.max_weight_matching(G, weight='weight')
#             print("Max_Match")
    
#             print(M_max)
            list_M_max = list(M_max)
            print(list_M_max)
            
    

#             print(allRouteDistance)
            for l in list_M_max:
#                 print(l)
#                 print(type)
                
#                 print(list(l))
#                 print(trip_df.trip_distance[list(l)[0]])
#                 print(trip_df.trip_distance[list(l)[1]])
                total_sum += trip_df.trip_distance[list(l)[0]]
                total_sum += trip_df.trip_distance[list(l)[1]]
                
                if allRouteDistance.get(l):
                    totalRouteDistance += round(float(allRouteDistance.get(l)), 2)
#                     print(round(float(allRouteDistance.get(l)), 2))
                else:
                    totalRouteDistance += round(float(allRouteDistance.get(l[::-1])), 2)
#                     print(round(float(allRouteDistance.get(l[::-1])), 2))
#                     totalRouteDistance += float(allRouteDistance.get(l[::-1]))

              

            
        
#             print(list_M_max)
            trips_merged.extend(list_M_max)
#             
#             trips_covered.add(list_M_max[0][0])
#             trips_covered.add(list_M_max[0][1])
#             total_sum += trip_df.trip_distance[list_M_max[0][0]]
#             total_sum += trip_df.trip_distance[list_M_max[0][1]]
            pools_processed += 1
        
        
        
#             print(trips_merged)
#             print(totalRouteDistance)
# print(pools_processed)
end_time = time.time()
# print(trips_merged)
    


    
    

[(16309, 16310), (16307, 16311), (16306, 16303), (16304, 16308)]
[(16320, 16315), (16322, 16316), (16317, 16319), (16314, 16323), (16321, 16324), (16312, 16313)]
[(16328, 16326), (16329, 16327)]
[(16332, 16333), (16330, 16331)]
[(16336, 16334), (16337, 16335)]
[(16340, 16341), (16338, 16342)]
[(16345, 16346), (16344, 16343)]
[(16352, 16350), (16347, 16348), (16353, 16349)]
[(16355, 16356)]
[(16363, 16358), (16362, 16357), (16361, 16364), (16360, 16359)]
[(16369, 16365), (16370, 16374), (16366, 16367), (16368, 16372), (16371, 16373)]
[(16377, 16378), (16376, 16379)]
[(16380, 16381), (16382, 16383)]
[(16385, 16387), (16384, 16386)]
[(16389, 16390), (16388, 16391)]
[(16392, 16393), (16394, 16395)]
[(16396, 16397), (16398, 16399)]
[(16405, 16406), (16402, 16404)]
[(16410, 16407), (16408, 16409)]
[(16411, 16412)]
[(16417, 16413), (16414, 16415)]
[(16419, 16420)]
[(16429, 16430), (16426, 16431), (16424, 16427), (16422, 16423), (16425, 16428)]
[(16433, 16435), (16432, 16434)]
[(16436, 16438)]

[(17393, 17394), (17388, 17389), (17396, 17391), (17395, 17390)]
[(17400, 17406), (17404, 17407), (17398, 17399), (17408, 17401), (17402, 17397), (17403, 17405)]
[(17410, 17413), (17411, 17414), (17409, 17412)]
[(17417, 17418), (17420, 17421), (17416, 17419), (17422, 17415)]
[(17428, 17429), (17424, 17430), (17426, 17423), (17425, 17427)]
[(17432, 17436), (17434, 17431), (17440, 17438), (17442, 17443), (17435, 17439), (17441, 17437)]
[(17449, 17453), (17452, 17444), (17456, 17450), (17448, 17451), (17454, 17447), (17445, 17455)]
[(17464, 17462), (17458, 17461), (17457, 17459), (17460, 17463)]
[(17474, 17469), (17468, 17470), (17472, 17476), (17465, 17478), (17466, 17471), (17473, 17479), (17475, 17477)]
[(17481, 17482), (17484, 17485), (17480, 17483), (17486, 17487)]
[(17489, 17491), (17496, 17501), (17492, 17494), (17498, 17495), (17497, 17493), (17499, 17500), (17488, 17490)]
[(17508, 17511), (17513, 17507), (17514, 17510), (17512, 17502), (17504, 17503), (17506, 17509)]
[(17521, 175

In [33]:
print(pools_processed)

148


In [34]:
distanceSavedPercent = ((total_sum - totalRouteDistance) * 100)/ all_trips_sum

In [35]:
# total_merged_trips = len(trips_covered)
# tripsSavedPercent = (total_merged_trips*100)/(2*total_trips)

In [36]:
total_merged_trips = 2 * len(trips_merged)
tripsSavedPercent = (total_merged_trips*100)/(2 * total_trips)

In [37]:
# dist_plot = [distanceSavedPercent,  ]
# trip_plot = [tripsSavedPercent, ]
total_trips


1290

In [38]:
distanceSavedPercent

47.838184895030025

In [39]:
total_merged_trips

1212

In [40]:
tripsSavedPercent

46

In [41]:
totalRouteDistance

10873.319999999994

In [42]:
total_sum

22101.649999999972

In [43]:
all_trips_sum

23471.47999999997

In [44]:
trips_merged

[(16309, 16310),
 (16307, 16311),
 (16306, 16303),
 (16304, 16308),
 (16320, 16315),
 (16322, 16316),
 (16317, 16319),
 (16314, 16323),
 (16321, 16324),
 (16312, 16313),
 (16328, 16326),
 (16329, 16327),
 (16332, 16333),
 (16330, 16331),
 (16336, 16334),
 (16337, 16335),
 (16340, 16341),
 (16338, 16342),
 (16345, 16346),
 (16344, 16343),
 (16352, 16350),
 (16347, 16348),
 (16353, 16349),
 (16355, 16356),
 (16363, 16358),
 (16362, 16357),
 (16361, 16364),
 (16360, 16359),
 (16369, 16365),
 (16370, 16374),
 (16366, 16367),
 (16368, 16372),
 (16371, 16373),
 (16377, 16378),
 (16376, 16379),
 (16380, 16381),
 (16382, 16383),
 (16385, 16387),
 (16384, 16386),
 (16389, 16390),
 (16388, 16391),
 (16392, 16393),
 (16394, 16395),
 (16396, 16397),
 (16398, 16399),
 (16405, 16406),
 (16402, 16404),
 (16410, 16407),
 (16408, 16409),
 (16411, 16412),
 (16417, 16413),
 (16414, 16415),
 (16419, 16420),
 (16429, 16430),
 (16426, 16431),
 (16424, 16427),
 (16422, 16423),
 (16425, 16428),
 (16433, 16435